In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import os

# Pandas Profiling

In [11]:
df = pd.read_excel('CitiesBySunshineDuration.xlsx')
profile = ProfileReport(df, title='Sunshine Data')
display(profile)

# Attaching data

In [30]:
from tqdm import tqdm
from geopy.geocoders import Nominatim

def find_coordinates_and_save():

    df = pd.read_excel('CitiesBySunshineDuration.xlsx')

    latitudes = []
    longitudes = []
    failed = []
    geolocator = Nominatim(user_agent='my-app-name')
    for country, city in tqdm(df[['Country', 'City']].iloc):
        try:
            try:
                # First try with explicitly setting country and city
                location = geolocator.geocode({'country': country, 'city': city})
                if location is None:
                    raise ValueError('')
            except:
                # If fail, pass as a string to be parsed
                location = geolocator.geocode(f'{city}, {country}')
                if location is None:
                    raise ValueError('')
        except:
            # Else report failed rows
            failed.append((country, city))
            location = {'latitude': None, 'longitude': None}
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    df['Latitude'] = latitudes
    df['Longitude'] = longitudes
    if len(failed) > 0:
        print(f'{len(failed)} coordinates not found for: {failed}')
    df.to_excel('CitiesBySunshineDurationWithCoordinates.xlsx', index=False)

find_coordinates_and_save()

1it [00:01,  1.04s/it]2021-08-15 15:32:59.892 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?country=Ivory+Coast&city=Bouak%C3%A9&format=json&limit=1
59it [00:32,  2.03it/s]2021-08-15 15:33:30.980 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?country=South+Sudan&city=Wau&format=json&limit=1
290it [02:33,  1.67it/s]2021-08-15 15:35:32.583 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', po

# Daylight hours!

In [33]:
import datetime as datetime
d1 = datetime.datetime(2021, 6, 1, 6)
d2 = datetime.datetime(2025, 7, 1, 7,1,1)
(d2 - d1)

datetime.timedelta(days=1491, seconds=3661)

In [37]:
datetime.datetime(1899,12,31)

datetime.datetime(1899, 12, 31, 0, 0)

In [12]:
df = pd.read_excel('NOAA_Solar_Calculations_day.xls')

In [83]:
def to_rads(n: float):
    return n / 360 * 2 * np.pi

def to_degs(n: float):
    return n / 2 / np.pi * 360

def decimal_to_hms(y):
    x = float(y)
    hour = x // (1/24)
    x -= hour * (1/24)
    minute = x // (1/24/60)
    x -= minute * (1/24/60)
    second = x // (1/24/60/60)
    x -= second * (1/24/60/60)
    return {'hour': int(hour), 'minute': int(minute), 'second': int(second)}

def solar_elevation(lat: float, long_: float, year: int, month: int, day: int, timezone: float):
    """
    Reproduces calculation for solar elevation as provided in
    https://gml.noaa.gov/grad/solcalc/calcdetails.html
    General > NOAA_Solar_Calculations_day.xls
    """
    # [D] Date
    date_as_excel_number = (datetime.datetime(year, month, day) - datetime.datetime(1899,12,30)).days
    # date_as_excel_number = (datetime.datetime(year, month, day) - datetime.datetime(1899, 12, 31)).days

    # [E] Time (past local midnight)
    time_past_midnight = 0.1/24

    # [F] Julian Day
    julian_day = date_as_excel_number + 2415018.5 + time_past_midnight - timezone/24

    # [G] Julian Century
    julian_century = (julian_day - 2451545)/36525

    # [I] Geom Mean Long Sun (deg)
    geom_mean_long_sun = (280.46646 + julian_century * (36000.76983 + julian_century * 0.0003032)) % 360

    # [J] Geom Mean Anom Sun (deg)
    geom_mean_anom_sun = 357.52911 + julian_century * (35999.05029 - 0.0001537 * julian_century)

    # [K] Eccent Earth Orbit
    eccent_earth_orbit = 0.016708634-julian_century*(0.000042037+0.0000001267*julian_century)

    # [L] Sun Eq of Ctr
    sun_eq_of_ctr = np.sin(to_rads(geom_mean_anom_sun))*(1.914602-julian_century*(0.004817+0.000014*julian_century))+np.sin(to_rads(2*geom_mean_anom_sun))*(0.019993-0.000101*julian_century)+np.sin(to_rads(3*geom_mean_anom_sun))*0.000289

    # [M] Sun True Long (deg)
    sun_true_long = geom_mean_long_sun + sun_eq_of_ctr

    # [N] Sun True Anom (deg)
    sun_true_anom = geom_mean_anom_sun + sun_eq_of_ctr

    # [O] Sun rad vector (AUs)
    sun_rad_vector = (1.000001018*(1-eccent_earth_orbit ** 2))/(1+eccent_earth_orbit*np.cos(to_rads(sun_true_anom)))

    # [P] Sun App Long (deg)
    sun_app_long = sun_true_long-0.00569-0.00478*np.sin(to_rads(125.04-1934.136*julian_century))

    # [Q] Mean Obliq Ecliptic (deg)
    mean_obliq_ecliptic = 23+(26+((21.448-julian_century*(46.815+julian_century*(0.00059-julian_century*0.001813))))/60)/60

    # [R] Obliq Corr (deg)
    obliq_corr = mean_obliq_ecliptic+0.00256*np.cos(to_rads(125.04-1934.136*julian_century))

    # [S] Sun Rt Ascen (deg)
    sun_rt_ascen = to_degs(np.arctan2(np.cos(to_rads(sun_app_long)),np.cos(to_rads(obliq_corr))*np.sin(to_rads(sun_app_long))))

    # [T] Sun Declin (deg)
    sun_declin = to_degs(np.arcsin(np.sin(to_rads(obliq_corr))*np.sin(to_rads(sun_app_long))))

    # [U] var y
    var_y = np.tan(to_rads(obliq_corr/2))*np.tan(to_rads(obliq_corr/2))

    # [V] eq of time (minutes)
    eq_of_time = 4*to_degs(var_y*np.sin(2*to_rads(geom_mean_long_sun))-2*eccent_earth_orbit*np.sin(to_rads(geom_mean_anom_sun))+4*eccent_earth_orbit*var_y*np.sin(to_rads(geom_mean_anom_sun))*np.cos(2*to_rads(geom_mean_long_sun))-0.5*var_y*var_y*np.sin(4*to_rads(geom_mean_long_sun))-1.25*eccent_earth_orbit*eccent_earth_orbit*np.sin(2*to_rads(geom_mean_anom_sun)))

    # [W] HA sunrise (deg)
    ha_sunrise = to_degs(np.arccos(np.cos(to_rads(90.833))/(np.cos(to_rads(lat))*np.cos(to_rads(sun_declin)))-np.tan(to_rads(lat))*np.tan(to_rads(sun_declin))))
    if np.isnan(ha_sunrise):
        ha_sunrise = 0

    # [X] Solar noon (LST)
    solar_noon = (720-4*long_-eq_of_time+timezone*60)/1440

    # [Y] Sunrise time (LST)
    sunrise_time = solar_noon-ha_sunrise*4/1440

    # [Z] Sunset time (LST)
    sunset_time = solar_noon+ha_sunrise*4/1440

    # [AA] Sunlight duration (minutes)
    sunlight_duration = 8 * ha_sunrise

    # [AB] True solar time (min)
    true_solar_time = (time_past_midnight*1440+eq_of_time+4*long_-60*timezone) % 1440

    # [AC] Hour angle (deg)
    hour_angle = (true_solar_time/4+180) if (true_solar_time<0) else (true_solar_time/4-180)

    # [AD] Solar Zenith Angle (deg)
    solar_zenith_angle = to_degs(np.arccos(np.sin(to_rads(lat))*np.sin(to_rads(sun_declin))+np.cos(to_rads(lat))*np.cos(to_rads(sun_declin))*np.cos(to_rads(hour_angle))))

    # [AE] Solar Elevation Angle (deg)
    solar_elevation_angle = 90 - solar_zenith_angle

    # [AF] Approx atmospheric refraction
    """
    IF(
        AE2>85,
        0,
        IF(
            AE2>5,
            58.1/TAN(RADIANS(AE2))-0.07/POWER(TAN(RADIANS(AE2)),3)+0.000086/POWER(TAN(RADIANS(AE2)),5),
            IF(
                AE2>-0.575,
                1735+AE2*(-518.2+AE2*(103.4+AE2*(-12.79+AE2*0.711))),
                -20.772/TAN(RADIANS(AE2))
            )
        )
    )/3600
    """
    if solar_elevation_angle > 85:
        approx_atmospheric_refraction = 0
    elif solar_elevation_angle > 5:
        approx_atmospheric_refraction = 58.1/np.tan(to_rads(solar_elevation_angle))-0.07/np.power(np.tan(to_rads(solar_elevation_angle)),3)+0.000086/np.power(np.tan(to_rads(solar_elevation_angle)),5)
    elif solar_elevation_angle > -0.575:
        approx_atmospheric_refraction = 1735+solar_elevation_angle*(-518.2+solar_elevation_angle*(103.4+solar_elevation_angle*(-12.79+solar_elevation_angle*0.711)))
    else:
        approx_atmospheric_refraction = -20.772/np.tan(to_rads(solar_elevation_angle))
    approx_atmospheric_refraction /= 3600

    # [AG] Solar elevation, refraction corrected (deg)
    solar_elevation_corrected = solar_elevation_angle + approx_atmospheric_refraction

    # [AH] Solar azimuth angle (deg)
    # solar_azimuth_angle = didn't bother with this one...

    results = {
        'sunrise': datetime.datetime(year, month, day, **decimal_to_hms(sunrise_time)),
        'sunset': datetime.datetime(year, month, day, **decimal_to_hms(sunset_time)),
        'sun_duration (hrs)': sunlight_duration/60,
    }
    return results

In [130]:
solar_elevation(-76.7899344,-1,2021,8,15,0)

{'sunrise': datetime.datetime(2021, 8, 15, 12, 8, 31),
 'sunset': datetime.datetime(2021, 8, 15, 12, 8, 34),
 'sun_duration (hrs)': 0.0007272516778463487}